# Merging the Structured and Unstructured Data Set following NLP analysis

This notebook takes the structured data set and cleans it to keep only the diagnoses which will be considered for the proof of concept, and matches a doctor's note for each patient.

Doctor's notes may be repeated depedning on the data set size limitations.

In [1]:
# Libraries
import pandas as pd
import numpy as np
import re

In [2]:
# Paths
docnotes_csv = 'output_comparison.csv' #Unstructured data set following information retrieval
ehr_csv = 'LengthOfStay.csv'#Structured Data set
cleaned_csv_path = 'MergedData.csv' #Where to save the combined structured + unstructured data sets

In [3]:
#First draft Erika
# Load doctor's notes
df = pd.read_csv(docnotes_csv)

# Kep required columns
notescols_tokeep = ['Part 7', 'Verdict - asthma', 'Verdict - iron def', 'Verdict - pneumoni', 'Verdict - substance', 'Verdict - fibrosis', 'Verdict - malnutr']
docnotes_df = df[notescols_tokeep]

# In the verdict column replace the word "Positive" with 1 and fill everything else with 0 - this will make the values comparable to the structured data set
verdict_columns = ['Verdict - asthma', 'Verdict - iron def', 'Verdict - pneumoni', 'Verdict - substance', 'Verdict - fibrosis', 'Verdict - malnutr']
docnotes_df[verdict_columns] = docnotes_df[verdict_columns].replace('Positive', 1).fillna(0)

# Print dataframe
# print(docnotes_df.head(3))

C:\Users\erika\AppData\Local\Temp\ipykernel_5772\2879214836.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docnotes_df[verdict_columns] = docnotes_df[verdict_columns].replace('Positive', 1).fillna(0)


In [4]:
# First draft Erika
# Load data from the structured csv file

df = pd.read_csv(ehr_csv)

# The POC focuses on these main diagnoses
diagnoses_columns = ['asthma', 'irondef', 'pneum', 'substancedependence', 'fibrosisandother', 'malnutrition']

# Filter rows that have at least one condition marked with 1
structured_df = df[df[diagnoses_columns].eq(1).any(axis=1)]

In [5]:
# First draft Erika
docnotes_df = docnotes_df.rename(columns=dict(zip(verdict_columns, diagnoses_columns)))

# create a new column to place the doctor's notes in
structured_df['Doc Note'] = np.nan

for index, row in structured_df.iterrows():
    # Extract the values from the current row in structured_df
    current_diagnoses_values = row[diagnoses_columns]

    # Check for an exact match in docnotes_df
    match_mask = (docnotes_df[diagnoses_columns].eq(current_diagnoses_values)).all(axis=1)

    # Copy a random note that matches the set of diagnoses
    if match_mask.any():
        matched_notes = docnotes_df.loc[match_mask, 'Part 7']
        selected_note = np.random.choice(matched_notes)
        structured_df.at[index, 'Doc Note'] = selected_note

# Print the updated structured_df
# print(structured_df.head(3))

C:\Users\erika\AppData\Local\Temp\ipykernel_5772\2769595952.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  structured_df['Doc Note'] = np.nan


In [6]:
# First draft Erika
# Filter out rows with blank 'Doc Note' column
structured_df = structured_df.dropna(subset=['Doc Note'])
# Remove the columns which are not needed
structured_df = structured_df.drop(columns=['vdate', 'secondarydiagnosisnonicd9', 'discharged', 'facid', 'lengthofstay', 'dialysisrenalendstage', 'psychologicaldisordermajor', 'depress', 'psychother', 'hemo'])

# Ensure that gender in the doc note is consistent with the gender in the gender column, to make the note make more sense
for index, row in structured_df.iterrows():
    # Extract values from the current row
    doc_note = row['Doc Note']
    gender = row['gender']

    # Check if gender contains 'F' and doc note contains 'male', then replace with 'female'
    if 'F' in gender and re.search(r'\bmale\b', doc_note, flags=re.IGNORECASE):
        structured_df.at[index, 'Doc Note'] = re.sub(r'\bmale\b', 'female', doc_note, flags=re.IGNORECASE)

    # Check if gender contains 'M' and doc note contains 'female', then replace with 'male'
    elif 'M' in gender and re.search(r'\bfemale\b', doc_note, flags=re.IGNORECASE):
        structured_df.at[index, 'Doc Note'] = re.sub(r'\bfemale\b', 'male', doc_note, flags=re.IGNORECASE)

In [7]:
# Save the cleaned DataFrame to a new CSV file
structured_df.to_csv(cleaned_csv_path, index=False)